# Lab-05 YOLOv7 Train

Please run the code with "VScode-devcontainer".

> You can find the tutorial provided by Visual Studio Code here :   
> [https://code.visualstudio.com/docs/devcontainers/containers](https://code.visualstudio.com/docs/devcontainers/containers)

## Import Required Libraries

In [32]:
from copy import deepcopy
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
import yaml
from torch.cuda import amp
from tqdm import tqdm

import test  # import test.py to get mAP after each epoch
from models.yolo import Model
from utils.autoanchor import check_anchors
from utils.datasets import LoadImagesAndLabels
from utils.general import labels_to_class_weights, increment_path, init_seeds, fitness, check_dataset, one_cycle, colorstr
from utils.loss import ComputeLoss, ComputeLossOTA
from utils.torch_utils import ModelEMA, intersect_dicts

## Hyper-Parameters

In [33]:
OPTIMIZER_NAME = "SGD"
BATCH_SIZE = 4
EPOCH = 55

## Open file

In [34]:
# Data dict
with open("data/AquariumDataset/data.yaml") as f:
    data_dict = yaml.load(f, Loader=yaml.SafeLoader)

# Hyperparameters
with open("data/hyp.scratch.custom.yaml") as f:
    hyp = yaml.load(f, Loader=yaml.SafeLoader)

## Get Device

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Prepare Dataset for fine-tune

- Dataset : Aquarium Dataset
    - [https://public.roboflow.com/object-detection/aquarium](https://public.roboflow.com/object-detection/aquarium)

---

- Dataloader
    - [pin_memory](https://pytorch.org/docs/stable/data.html#memory-pinning)
    - [Train Custom Data (From YOLOv5)](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data)

---

```
wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
```

In [37]:
def create_dataloader(path, batch_size, augment=False, pad=0.0, rect=False, prefix=""):
    dataset = LoadImagesAndLabels(path=path, img_size=640, batch_size=batch_size, augment=augment, hyp=hyp, rect=rect, pad=pad, prefix=prefix)  # augment images  # augmentation hyperparameters  # rectangular training
    dataloader = DataLoader(dataset, batch_size=min(batch_size, len(dataset)), pin_memory=True, collate_fn=LoadImagesAndLabels.collate_fn)

    return dataloader, dataset


# Trainloader
dataloader, dataset = create_dataloader(path=data_dict["train"], batch_size=BATCH_SIZE, augment=True, prefix=colorstr("train: "))
testloader = create_dataloader(path=data_dict["val"], batch_size=BATCH_SIZE * 2, rect=True, pad=0.5, prefix=colorstr("val: "))[0]  # testloader

# check
check_dataset(data_dict)

train: Scanning 'data/AquariumDataset/train/labels.cache' images and labels... 448 found, 0 missing, 1 empty, 0 corrupted: 100%|██████████| 448/448 [00:00<?, ?it/s]
val: Scanning 'data/AquariumDataset/valid/labels.cache' images and labels... 127 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 127/127 [00:00<?, ?it/s]


## Download pre-trained weight

Attempt to download the file if it does not exist.  

- Reference  
    - [https://github.com/WongKinYiu/yolov7/releases/tag/v0.1](https://github.com/WongKinYiu/yolov7/releases/tag/v0.1)

In [38]:
def attempt_download(filename):
    file = Path(filename)

    # download if not found locally
    if not file.exists():
        repo = "WongKinYiu/yolov7"
        tag = "v0.1"
        name = file.name
        url = f"https://github.com/{repo}/releases/download/{tag}/{name}"
        print(f"Downloading {url} to {file}...")
        torch.hub.download_url_to_file(url, file)


attempt_download(filename="yolov7_training.pt")

## Model

In [39]:
ckpt = torch.load("yolov7_training.pt", map_location=device)  # load checkpoint
model = Model("cfg/training/yolov7.yaml", nc=data_dict["nc"], anchors=hyp.get("anchors")).to(device)  # create
state_dict = ckpt["model"].float().state_dict()  # to FP32
state_dict = intersect_dicts(state_dict, model.state_dict(), exclude=["anchor"])  # intersect
model.load_state_dict(state_dict, strict=False)  # load
print("Transferred %g/%g items" % (len(state_dict), len(model.state_dict())))  # report

# Freeze
freeze = [f"model.{x}." for x in range(0)]  # parameter names to freeze (full or partial)
for k, v in model.named_parameters():
    v.requires_grad = True  # train all layers
    if any(x in k for x in freeze):
        print("freezing %s" % k)
        v.requires_grad = False

# Model parameters
nl = model.model[-1].nl  # number of detection layers (used for scaling hyp['obj'])
model.nc = data_dict["nc"]  # attach number of classes to model
model.names = data_dict["names"]
model.hyp = hyp  # attach hyperparameters to model
model.gr = 1.0  # iou loss ratio (obj_loss = 1.0 or iou)
model.class_weights = labels_to_class_weights(dataset.labels, model.nc).to(device) * model.nc  # attach class weights
hyp["box"] *= 3.0 / nl  # scale to layers
hyp["cls"] *= model.nc / 80.0 * 3.0 / nl  # scale to classes and layers
hyp["obj"] *= (640 / 640) ** 2 * 3.0 / nl  # scale to image size and layers
hyp["label_smoothing"] = 0.0

Transferred 555/566 items


## Optimizer

In [40]:
accumulate = max(round(64 / BATCH_SIZE), 1)  # accumulate loss before optimizing
hyp["weight_decay"] *= BATCH_SIZE * accumulate / 64  # scale weight_decay
print(f"Scaled weight_decay = {hyp['weight_decay']}")

pg0, pg1, pg2 = [], [], []  # optimizer parameter groups
for k, v in model.named_modules():
    if hasattr(v, "bias") and isinstance(v.bias, nn.Parameter):
        pg2.append(v.bias)  # biases
    if isinstance(v, nn.BatchNorm2d):
        pg0.append(v.weight)  # no decay
    elif hasattr(v, "weight") and isinstance(v.weight, nn.Parameter):
        pg1.append(v.weight)  # apply decay
    if hasattr(v, "im"):
        if hasattr(v.im, "implicit"):
            pg0.append(v.im.implicit)
        else:
            for iv in v.im:
                pg0.append(iv.implicit)
    if hasattr(v, "imc"):
        if hasattr(v.imc, "implicit"):
            pg0.append(v.imc.implicit)
        else:
            for iv in v.imc:
                pg0.append(iv.implicit)
    if hasattr(v, "imb"):
        if hasattr(v.imb, "implicit"):
            pg0.append(v.imb.implicit)
        else:
            for iv in v.imb:
                pg0.append(iv.implicit)
    if hasattr(v, "imo"):
        if hasattr(v.imo, "implicit"):
            pg0.append(v.imo.implicit)
        else:
            for iv in v.imo:
                pg0.append(iv.implicit)
    if hasattr(v, "ia"):
        if hasattr(v.ia, "implicit"):
            pg0.append(v.ia.implicit)
        else:
            for iv in v.ia:
                pg0.append(iv.implicit)
    if hasattr(v, "attn"):
        if hasattr(v.attn, "logit_scale"):
            pg0.append(v.attn.logit_scale)
        if hasattr(v.attn, "q_bias"):
            pg0.append(v.attn.q_bias)
        if hasattr(v.attn, "v_bias"):
            pg0.append(v.attn.v_bias)
        if hasattr(v.attn, "relative_position_bias_table"):
            pg0.append(v.attn.relative_position_bias_table)
    if hasattr(v, "rbr_dense"):
        if hasattr(v.rbr_dense, "weight_rbr_origin"):
            pg0.append(v.rbr_dense.weight_rbr_origin)
        if hasattr(v.rbr_dense, "weight_rbr_avg_conv"):
            pg0.append(v.rbr_dense.weight_rbr_avg_conv)
        if hasattr(v.rbr_dense, "weight_rbr_pfir_conv"):
            pg0.append(v.rbr_dense.weight_rbr_pfir_conv)
        if hasattr(v.rbr_dense, "weight_rbr_1x1_kxk_idconv1"):
            pg0.append(v.rbr_dense.weight_rbr_1x1_kxk_idconv1)
        if hasattr(v.rbr_dense, "weight_rbr_1x1_kxk_conv2"):
            pg0.append(v.rbr_dense.weight_rbr_1x1_kxk_conv2)
        if hasattr(v.rbr_dense, "weight_rbr_gconv_dw"):
            pg0.append(v.rbr_dense.weight_rbr_gconv_dw)
        if hasattr(v.rbr_dense, "weight_rbr_gconv_pw"):
            pg0.append(v.rbr_dense.weight_rbr_gconv_pw)
        if hasattr(v.rbr_dense, "vector"):
            pg0.append(v.rbr_dense.vector)

if OPTIMIZER_NAME == "Adam":
    optimizer = optim.Adam(pg0, lr=hyp["lr0"], betas=(hyp["momentum"], 0.999))  # adjust beta1 to momentum
elif OPTIMIZER_NAME == "SGD":
    optimizer = optim.SGD(pg0, lr=hyp["lr0"], momentum=hyp["momentum"], nesterov=True)

optimizer.add_param_group({"params": pg1, "weight_decay": hyp["weight_decay"]})  # add pg1 with weight_decay
optimizer.add_param_group({"params": pg2})  # add pg2 (biases)
print("Optimizer groups: %g .bias, %g conv.weight, %g other" % (len(pg2), len(pg1), len(pg0)))
del pg0, pg1, pg2

Scaled weight_decay = 0.0005
Optimizer groups: 95 .bias, 95 conv.weight, 98 other


## Define Train Function

In [41]:
def train():
    global ckpt, state_dict

    print(colorstr("hyperparameters: ") + ", ".join(f"{k}={v}" for k, v in hyp.items()))
    save_dir = Path(increment_path(Path("runs/train") / "yolov7-custom", exist_ok=False))

    # Directories
    wdir = save_dir / "weights"
    wdir.mkdir(parents=True, exist_ok=True)  # make dir
    weight_pt = wdir / "weight.pt"
    results_file = save_dir / "results.txt"

    # Save run settings
    with open(save_dir / "hyp.yaml", "w") as f:
        yaml.dump(hyp, f, sort_keys=False)

    # Init
    best_fitness = 0.0
    init_seeds(1)

    # Scheduler
    lf = one_cycle(1, hyp["lrf"], EPOCH)  # cosine 1->hyp['lrf']
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)

    # EMA
    ema = ModelEMA(model)
    if ckpt.get("ema"):
        ema.ema.load_state_dict(ckpt["ema"].float().state_dict())
        ema.updates = ckpt["updates"]

    # Results
    if ckpt.get("training_results") is not None:
        results_file.write_text(ckpt["training_results"])  # write results.txt

    # Number of batches
    nb = len(dataloader)

    # Anchors
    check_anchors(dataset, model=model, thr=hyp["anchor_t"], imgsz=640)
    model.half().float()  # pre-reduce anchor precision

    # Start training
    nw = max(round(hyp["warmup_epochs"] * nb), 1000)  # number of warmup iterations, max(3 epochs, 1k iterations)
    results = (0, 0, 0, 0, 0, 0, 0)  # P, R, mAP@.5, mAP@.5-.95, val_loss(box, obj, cls)
    scheduler.last_epoch = -1  # do not move
    scaler = amp.GradScaler()
    compute_loss_ota = ComputeLossOTA(model)  # init loss class
    compute_loss = ComputeLoss(model)  # init loss class

    print(f"Logging results to {save_dir}")
    print(f"Starting training for {EPOCH} epochs...")

    for epoch in range(EPOCH):
        model.train()

        mloss = torch.zeros(4, device=device)  # mean losses
        print(("\n" + "%10s" * 8) % ("Epoch", "gpu_mem", "box", "obj", "cls", "total", "labels", "img_size"))
        pbar = tqdm(enumerate(dataloader), total=nb)  # progress bar
        optimizer.zero_grad()

        # batch -------------------------------------------------------------
        for i, (imgs, targets, paths, _) in pbar:
            ni = i + nb * epoch  # number integrated batches (since train start)
            imgs = imgs.to(device, non_blocking=True).float() / 255.0  # uint8 to float32, 0-255 to 0.0-1.0

            # Warmup
            if ni <= nw:
                xi = [0, nw]  # x interp
                accumulate = max(1, np.interp(ni, xi, [1, 64 / BATCH_SIZE]).round())
                for j, x in enumerate(optimizer.param_groups):
                    # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                    x["lr"] = np.interp(ni, xi, [hyp["warmup_bias_lr"] if j == 2 else 0.0, x["initial_lr"] * lf(epoch)])
                    if "momentum" in x:
                        x["momentum"] = np.interp(ni, xi, [hyp["warmup_momentum"], hyp["momentum"]])

            # Forward
            with amp.autocast():
                pred = model(imgs)  # forward
                if "loss_ota" not in hyp or hyp["loss_ota"] == 1:
                    loss, loss_items = compute_loss_ota(pred, targets.to(device), imgs)  # loss scaled by batch_size
                else:
                    loss, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size

            # Backward
            scaler.scale(loss).backward()

            # Optimize
            if ni % accumulate == 0:
                scaler.step(optimizer)  # optimizer.step
                scaler.update()
                optimizer.zero_grad()
                if ema:
                    ema.update(model)

            # Print
            mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
            mem = "%.3gG" % (torch.cuda.memory_reserved() / 1e9 if torch.cuda.is_available() else 0)  # (GB)
            s = ("%10s" * 2 + "%10.4g" * 6) % ("%g/%g" % (epoch, EPOCH - 1), mem, *mloss, targets.shape[0], imgs.shape[-1])
            pbar.set_description(s)

        # Scheduler
        scheduler.step()

        # mAP
        ema.update_attr(model, include=["yaml", "nc", "hyp", "gr", "names", "stride", "class_weights"])
        final_epoch = epoch + 1 == EPOCH
        results, maps, times = test.test(data_dict, batch_size=BATCH_SIZE * 2, imgsz=640, model=ema.ema, dataloader=testloader, save_dir=save_dir, verbose=final_epoch, plots=final_epoch, compute_loss=compute_loss)

        # Write
        with open(results_file, "a") as f:
            f.write(s + "%10.4g" * 7 % results + "\n")  # append metrics, val_loss

        # Update best mAP
        fi = fitness(np.array(results).reshape(1, -1))  # weighted combination of [P, R, mAP@.5, mAP@.5-.95]
        if fi > best_fitness:
            best_fitness = fi
            log = {"epoch": epoch, "best_fitness": best_fitness, "training_results": results_file.read_text(), "model": deepcopy(model).half(), "ema": deepcopy(ema.ema).half(), "updates": ema.updates, "optimizer": optimizer.state_dict(), "wandb_id": None}
            torch.save(log, weight_pt)
            del log

    torch.cuda.empty_cache()
    return results

## Train

In [42]:
train()

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.026249999999999992, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.2, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0, paste_in=0.0, loss_ota=1, label_smoothing=0.0

autoanchor: Analyzing anchors... anchors/target = 5.03, Best Possible Recall (BPR) = 0.9991
Logging results to runs/train/yolov7-custom13
Starting training for 55 epochs...

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size


  0%|          | 0/112 [00:00<?, ?it/s]

      0/54     3.51G   0.07841   0.02561   0.03247    0.1365        78       640: 100%|██████████| 112/112 [00:26<00:00,  4.30it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 16/16 [00:02<00:00,  7.34it/s]


                 all         127         909     0.00503      0.0358    0.000544     0.00011

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size


      1/54     3.61G   0.06385    0.0258   0.02769    0.1173        48       640: 100%|██████████| 112/112 [00:26<00:00,  4.30it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 16/16 [00:02<00:00,  7.52it/s]


...

     53/54     4.19G   0.02294   0.01303  0.001303   0.03728        71       640: 100%|██████████| 112/112 [00:25<00:00,  4.36it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 16/16 [00:02<00:00,  7.80it/s]


                 all         127         909       0.837       0.778       0.826       0.526

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size


     54/54     4.19G     0.023   0.01313  0.001713   0.03784        46       640: 100%|██████████| 112/112 [00:25<00:00,  4.36it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 16/16 [00:02<00:00,  7.33it/s]


                 all         127         909       0.857       0.767       0.824       0.528
                fish         127         459       0.881       0.789        0.87       0.533
           jellyfish         127         155       0.906       0.916       0.952       0.569
             penguin         127         104       0.714       0.769       0.753       0.346
              puffin         127          74       0.764       0.656       0.691       0.341
               shark         127          57       0.893       0.649       0.767       0.576
            starfish         127          27           1       0.801       0.901       0.683
            stingray         127          33       0.839       0.788       0.834       0.649


(0.8566185969461217,
 0.766881042989147,
 0.8240403466043548,
 0.5282587715010063,
 0.05249493569135666,
 0.0471758209168911,
 0.0097836097702384)